In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
from scipy.stats import skew 
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KDTree
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import BallTree
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


import os
print(os.listdir())

import warnings
warnings.filterwarnings('ignore')

['house_price_competition_day_2_failed_trying.ipynb', 'house_price_advanced_view.ipynb', 'addr_kmeans.pkl', 'submission.csv', 'house_price_advanced_models.ipynb', 'my_model_submission.csv1', 'my_model_submission4.csv', 'house_price', 'addr_umap.pkl', 'Day1.ipynb', 'titanic', 'house-prices-advanced-regression-techniques.zip', 'titanic.zip', 'my_model_submission3.csv', 'house_price_competition_view_day2.ipynb', 'Untitled.ipynb', 'my_model_submission0.csv', 'X_umap.npy', 'addr_tfidf.pkl', 'Day2 Housing Price.ipynb', 'pca_model.pkl', 'my_model_submission.csv', 'predictions.csv', 'umap_model.pkl', 'predictions4.csv', 'my_model_submission1.csv', 'competition_day7.ipynb', 'Untitled1.ipynb', 'house_price_view.ipynb', 'home-data-for-ml-course.zip', '.ipynb_checkpoints', 'home-data-for-ml-course', 'house_price.ipynb', 'predictions3.csv', 'my_model_submission2.csv', 'best_interval_model.pth', 'predictions2.csv']


In [2]:
train = pd.read_csv('house_price/dataset.csv')
test = pd.read_csv('house_price/test.csv')
print ("Data is loaded!")

Data is loaded!


In [3]:
quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
quantitative.remove('sale_price')
quantitative.remove('id')
qualitative = [f for f in train.columns if train.dtypes[f] == 'object']

In [4]:
def dataset_fill_null(obj):
    obj['subdivision'].fillna('Unknown', inplace=True)
    obj.drop(columns=['sale_nbr'], inplace=True)
    obj['submarket'].fillna('Unknown', inplace=True)

missing = train.isnull().sum()
missing = missing[missing > 0]
print(missing)

dataset_fill_null(train)
dataset_fill_null(test)
print(train.shape)
print(test.shape)

sale_nbr       42182
subdivision    17550
submarket       1717
dtype: int64
(200000, 46)
(200000, 45)


In [5]:
# 构造原始地址字段
train_ID = train['id']
test_ID = test['id']
# Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)
# Deleting outliers
train.reset_index(drop=True, inplace=True)
# We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["sale_price"] = np.log1p(train["sale_price"])
y = train.sale_price.reset_index(drop=True)

In [6]:
def preprocess_and_encode(df, y=None, drop_high_card=True):
    df = df.copy()
    
    # ================= 清洗阶段 ================= #
    addresses = (df['city'].fillna('') + ' ' + df['subdivision'].fillna('')).str.lower()
    
    # 分词（空格切）
    # 用 CountVectorizer 可以保留频率稀疏性
    
    vectorizer = CountVectorizer(
        max_features=1000,         # 可调大小
        stop_words=None,      # 去常见词
        token_pattern=r'\b\w+\b',  # 标准单词
        ngram_range=(1, 2)         # 一元和二元组都试试
    )
    address_vecs = vectorizer.fit_transform(addresses)
    
    address_df = pd.DataFrame(address_vecs.toarray(), columns=vectorizer.get_feature_names_out())
    address_df.index = df.index
    df = pd.concat([df, address_df], axis=1)
    df = df.drop(columns=['city', 'subdivision'])

    # 使用同一个 address_vecs（CountVectorizer 输出）
    svd = TruncatedSVD(n_components=50, random_state=42)
    address_pca = svd.fit_transform(address_vecs)
    
    # 转成 DataFrame 并拼接
    pca_df = pd.DataFrame(address_pca, columns=[f'address_pca_{i+1}' for i in range(address_pca.shape[1])], index=df.index)
    df = pd.concat([df, pca_df], axis=1)

    # 补充缺失类别为 'None'
    cat_cols = df.select_dtypes(include=['object', 'category', 'string']).columns
    
    for col in cat_cols:
        df[col] = df[col].fillna('None')

    # 填充数值缺失为 0
    num_cols = df.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        df[col] = df[col].fillna(0)

    # 拆解日期特征
    if 'sale_date' in df.columns:
        df['sale_date'] = pd.to_datetime(df['sale_date'])
        df['sale_year'] = pd.to_datetime(df['sale_date']).dt.year
        df['sale_month'] = pd.to_datetime(df['sale_date']).dt.month
        df['house_age'] = df['sale_year'] - df['year_built']
        df['reno_age'] = df['sale_year'] - df['year_reno']
        df['has_reno'] = (df['year_reno'] > 0).astype(int)
        df['land_imp_ratio'] = df['land_val'] / (df['imp_val'] + 1e-5)
        df['time_since_sale'] = (df['sale_date'].max() - df['sale_date']).dt.days / 30  # 单位：月        
        df = df.drop(columns='sale_date')





    # ================= 编码阶段 ================= #
    cat_cols = df.select_dtypes(include='object').columns
    low_card_cols = []
    high_card_cols = []
    
    for col in cat_cols:
        try:
            n_unique = df[col].nunique()
            if isinstance(n_unique, (int, np.integer)):
                if n_unique <= 50:
                    low_card_cols.append(col)
                else:
                    high_card_cols.append(col)
            else:
                print(f"[跳过] {col} 的 nunique 结果不是标量: {n_unique}")
        except Exception as e:
            print(f"[异常] {col}: {e}")


    # One-hot 编码
    X_cat = pd.get_dummies(df[low_card_cols], dummy_na=True)

    # Target 编码
    if y is not None and high_card_cols:
        encoder = ce.TargetEncoder()
        X_target = encoder.fit_transform(df[high_card_cols], y)
        X_target.columns = [f"{col}_te" for col in high_card_cols]
    else:
        X_target = pd.DataFrame(index=df.index)

    # 数值标准化
    num_cols = df.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    X_num_scaled = pd.DataFrame(scaler.fit_transform(df[num_cols]), columns=num_cols, index=df.index)

    # 拼接所有
    X_final_df = pd.concat([X_num_scaled, X_cat, X_target], axis=1)

    return X_final_df


In [7]:
def add_knn_price_features_radius(
    df, base_df, lat_col='latitude', lon_col='longitude',
    target_col='sale_price', radii=[0.005, 0.01, 0.02], max_neighbors=300
):
    df = df.copy()
    coords_query = df[[lat_col, lon_col]].astype(np.float64).values
    coords_base = base_df[[lat_col, lon_col]].astype(np.float64).values
    targets_base = base_df[target_col].astype(np.float64).values

    tree = KDTree(coords_base, leaf_size=40, metric='euclidean')

    for r in radii:
        neighbor_indices = tree.query_radius(coords_query, r=r)
        
        stats = {
            'mean': [], 'std': [], 'min': [], 'max': [],
            'range': [], 'iqr': [], 'median': [], 'count': []
        }

        for i, inds in enumerate(neighbor_indices):
            if base_df is df:
                inds = inds[inds != i]

            if len(inds) == 0:
                for key in stats:
                    stats[key].append(np.nan if key != 'count' else 0)
            else:
                if len(inds) > max_neighbors:
                    inds = inds[:max_neighbors]
                vals = targets_base[inds]
                stats['mean'].append(np.mean(vals))
                stats['std'].append(np.std(vals))
                stats['min'].append(np.min(vals))
                stats['max'].append(np.max(vals))
                stats['range'].append(np.max(vals) - np.min(vals))
                stats['iqr'].append(np.percentile(vals, 75) - np.percentile(vals, 25))
                stats['median'].append(np.median(vals))
                stats['count'].append(len(inds))

        for key in stats:
            df[f'knn_radius_{key}_r{r}'] = stats[key]

    return df


def add_knn_price_features(df, base_df=None, lat_col='latitude', lon_col='longitude',
                           target_col='target', ks=[5, 10, 20]):
    if base_df is None:
        base_df = df  # 默认自己为近邻池

    coords_query = df[[lat_col, lon_col]].values
    coords_base = base_df[[lat_col, lon_col]].values
    tree = KDTree(coords_base, metric='euclidean')

    base_targets = base_df[target_col].values
    knn_features = {}

    for k in ks:
        dists, indices = tree.query(coords_query, k=k)
        neighbor_targets = base_targets[indices]

        knn_features[f'knn_price_mean_{k}'] = neighbor_targets.mean(axis=1)
        knn_features[f'knn_price_std_{k}'] = neighbor_targets.std(axis=1)
        knn_features[f'knn_price_range_{k}'] = neighbor_targets.max(axis=1) - neighbor_targets.min(axis=1)

    for col, val in knn_features.items():
        df[col] = val

    return df


In [8]:
# 应用预处理
# 对训练集（自己做自己）
train = add_knn_price_features(train, base_df=train, target_col='sale_price')
# 对测试集（用训练集做 base）
test = add_knn_price_features(test, base_df=train, target_col='sale_price')

train = add_knn_price_features_radius(train, base_df=train, target_col='sale_price')
# 对测试集（用训练集做 base）
test = add_knn_price_features_radius(test, base_df=train, target_col='sale_price')

train_features = train.drop(['sale_price'], axis=1)
test_features = test

features = pd.concat([train_features, test_features]).reset_index(drop=True)
print(features.shape)
print([col for col in features.columns if 'knn' in col])

X_final = preprocess_and_encode(features)

(400000, 77)
['knn_price_mean_5', 'knn_price_std_5', 'knn_price_range_5', 'knn_price_mean_10', 'knn_price_std_10', 'knn_price_range_10', 'knn_price_mean_20', 'knn_price_std_20', 'knn_price_range_20', 'knn_radius_mean_r0.005', 'knn_radius_std_r0.005', 'knn_radius_min_r0.005', 'knn_radius_max_r0.005', 'knn_radius_range_r0.005', 'knn_radius_iqr_r0.005', 'knn_radius_median_r0.005', 'knn_radius_count_r0.005', 'knn_radius_mean_r0.01', 'knn_radius_std_r0.01', 'knn_radius_min_r0.01', 'knn_radius_max_r0.01', 'knn_radius_range_r0.01', 'knn_radius_iqr_r0.01', 'knn_radius_median_r0.01', 'knn_radius_count_r0.01', 'knn_radius_mean_r0.02', 'knn_radius_std_r0.02', 'knn_radius_min_r0.02', 'knn_radius_max_r0.02', 'knn_radius_range_r0.02', 'knn_radius_iqr_r0.02', 'knn_radius_median_r0.02', 'knn_radius_count_r0.02']


In [9]:
X = X_final.iloc[:len(train_features)].reset_index(drop=True)
X_sub = X_final.iloc[len(train_features):].reset_index(drop=True)
print('X', X.shape, 'y', y.shape, 'X_sub', X_sub.shape)

protected_cols = []
overfit = [i for i in X.columns if i not in protected_cols and X[i].value_counts().iloc[0] / len(X) * 100 > 99.94]
print(overfit)

X = X.drop(overfit, axis=1)
X_sub = X_sub.drop(overfit, axis=1)
print('X', X.shape, 'y', y.shape, 'X_sub', X_sub.shape)

X (200000, 1156) y (200000,) X_sub (200000, 1156)
['join_status_nan', 'submarket_nan']
X (200000, 1154) y (200000,) X_sub (200000, 1154)


In [120]:
# ========== 1. 定义网络结构 ==========
class QuantileRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.2),

            nn.Linear(256, 128),
            nn.ReLU(),

            nn.Linear(128, 2),  # 可以加入新层

        )

    def forward(self, x):
        return self.net(x)



# ========== 2. Quantile Loss (Pinball Loss) ==========
def pinball_interval_loss(preds, target, alpha=0.8, coverage=1 ,penalty_weight=10.0):
    y_low = preds[:, 0]
    y_high = preds[:, 1]
    y = target.squeeze()

    # 覆盖惩罚
    under = (y < y_low).float()
    over = (y > y_high).float()
    if coverage < 0.95:
        #cover =((1-coverage)*250)
        cover = 1.3

    else: 
        cover = 1

    #cover =((1-coverage)*20) + 1

        
        
    miss_penalty = (2.0 / (1 - alpha)) * ((y_low - y) * under + (y - y_high) * over) * cover

    # 区间宽度惩罚
    width_penalty = (y_high - y_low)

    # 结构惩罚项：low > high 是不合法的
    structure_penalty = torch.mean(torch.relu(y_low - y_high)) * penalty_weight

    return torch.mean(width_penalty + miss_penalty) + structure_penalty


def interval_score_loss(preds, target, alpha=0.8):
    y_low = preds[:, 0]
    y_high = preds[:, 1]
    y = target.squeeze()

    width = y_high - y_low
    below = (y < y_low).float()
    above = (y > y_high).float()

    under_penalty = (2 / alpha) * (y_low - y) * below
    over_penalty = (2 / alpha) * (y - y_high) * above

    score = width + under_penalty + over_penalty
    return torch.mean(score)



# ========== 3. 训练函数 ==========
def train_model(X, y, alpha=0.8, epochs=200, batch_size=1024, lr=1e-3):
    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 数据准备
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, num_workers=4, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size)

    # 模型定义
    model = QuantileRegressor(input_dim=X.shape[1]).to(device)

    #optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-5)
    #scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)


    best_val_loss = float('inf')
    best_model = None
    patience, patience_counter = 10, 0

    # ========== 4. 训练循环 ==========
    coverage = 1
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = pinball_interval_loss(preds, yb, alpha, coverage)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            

        # 验证
        model.eval()
        with torch.no_grad():
            val_loss = 0
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                loss = pinball_interval_loss(preds, yb, alpha, coverage)
                val_loss += loss.item()


        print(f"Epoch {epoch+1}, Train Loss: {total_loss:.4f}, Val Loss: {val_loss:.4f}")

        with torch.no_grad():
            preds = model(X_val_tensor.to(device)).cpu().numpy()
            y_true = y_val_tensor.cpu().numpy()
            coverage = np.mean((y_true >= preds[:, 0]) & (y_true <= preds[:, 1]))
            avg_width = np.mean(preds[:, 1] - preds[:, 0])
            print(f"Val Coverage: {coverage:.4f}, Avg Width: {avg_width:.2f}")

        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Current LR: {current_lr:.6f}")

        
        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

    model.load_state_dict(best_model)
    torch.save(best_model, 'best_interval_model.pth')
    return model


# ========== 5. 用法示例 ==========
# X_final: numpy 数组 (N, D)
# y_train_lower, y_train_upper: numpy 数组 (N,)
# 合并成 target 矩阵：
# y_train = np.stack([y_train_lower, y_train_upper], axis=1)

# model = train_model(X_final, y_train)

# 输出预测
# model(torch.tensor(X_test).to(device)).cpu().detach().numpy()

In [121]:
X = X.astype(np.float32)
y = y.astype(np.float32)
X_sub = X_sub.astype(np.float32)

In [122]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math

model = train_model(X.values, y.values, epochs=150)
print("Finished training")

Epoch 1, Train Loss: 4366.3718, Val Loss: 49.5555
Val Coverage: 0.9823, Avg Width: 2.33
Current LR: 0.000377
Epoch 2, Train Loss: 538.3723, Val Loss: 38.7844
Val Coverage: 0.9894, Avg Width: 1.89
Current LR: 0.000422
Epoch 3, Train Loss: 491.4649, Val Loss: 36.0798
Val Coverage: 0.9827, Avg Width: 1.74
Current LR: 0.000432
Epoch 4, Train Loss: 505.0483, Val Loss: 40.9931
Val Coverage: 0.9923, Avg Width: 2.02
Current LR: 0.000413
Epoch 5, Train Loss: 473.4961, Val Loss: 40.5140
Val Coverage: 0.9950, Avg Width: 2.01
Current LR: 0.000415
Epoch 6, Train Loss: 477.9582, Val Loss: 38.6287
Val Coverage: 0.9926, Avg Width: 1.91
Current LR: 0.000423
Epoch 7, Train Loss: 449.5636, Val Loss: 36.9536
Val Coverage: 0.9919, Avg Width: 1.83
Current LR: 0.000429
Epoch 8, Train Loss: 453.4338, Val Loss: 43.7710
Val Coverage: 0.9899, Avg Width: 2.17
Current LR: 0.000402
Epoch 9, Train Loss: 451.8171, Val Loss: 38.4587
Val Coverage: 0.9953, Avg Width: 1.91
Current LR: 0.000423
Epoch 10, Train Loss: 451.8

In [115]:
X_sub = X_sub.astype(np.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pred = model(torch.tensor(X_sub.values).to(device)).cpu().detach().numpy()
print("Finished prediction")

Finished prediction


In [116]:
df_pred = pd.DataFrame(np.expm1(pred), columns=["pi_lower", "pi_upper"])
df_pred.insert(0, "id", test_ID)
df_pred.to_csv("predictions4.csv", index=False)


In [15]:
print(4096*2)


8192


In [16]:
torch.backends.cudnn.benchmark = True
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


NameError: name 'train_dataset' is not defined